In [116]:
import nbformat as nbf
import shutil



In [40]:
nb = nbf.read('testing_notebook.ipynb', as_version=4)

In [141]:
def get_inputs():
    table_name = input("table name (no spaces): ")
    while len(table_name.split())>1:
        table_name = input("table name (no spaces): ")
    number = input("number: ")
    desc = input("table description: ")
    temp_flag = ""
    while temp_flag not in ('T','F'):
        temp_flag = input("Temporary table T/F: ").upper()
    if temp_flag == 'T':
        temp_flag = True
    else:
        temp_flag = False
    return table_name,number,desc,temp_flag

In [142]:
def create_backup():
    original = r'testing_notebook.ipynb'
    target = r'copy_of_testing_notebook.ipynb'
    shutil.copyfile(original, target)

In [156]:
def new_title_cell(number,name,desc):
    cell = nbf.v4.new_markdown_cell(f"## {number}. {name} table - {desc} <a name={name}></a>")
    return [cell]

In [157]:
def new_drop_table_cells(name):
    drop_mk_cell = nbf.v4.new_markdown_cell(f"### Drop the {name} table if it already exists and remove its data from the S3 bucket")
    
    drop_code = f"""drop_{name} = \"DROP TABLE IF EXISTS fcsq.{name}\"
pydb.start_query_execution_and_wait(drop_{name})
bucket.objects.filter(Prefix=\"fcsq_processing/Adoption/{name}\").delete();"""    
    drop_code_cell = nbf.v4.new_code_cell(drop_code)
    
    return [drop_mk_cell,drop_code_cell]

In [158]:
def new_create_table_cells(name,code,temp_flag):
    create_mk_cell = nbf.v4.new_markdown_cell(f"### Create the {name} table")
    start_code = f"create_{name}_table =f\"\"\""
    if temp_flag:
        end_code = f"pydb.create_temp_table(create_{name}_table)"
    else:
        start_code += f"""
CREATE TABLE IF NOT EXISTS fcsq.{name}
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/{name}') AS"""
        end_code =  f"pydb.start_query_execution_and_wait(create_{name}_table);"
    
    create_code = start_code+code+"\"\"\"\n"+end_code 
    create_code_cell = nbf.v4.new_code_cell(create_code)
    
    return [create_mk_cell,create_code_cell]

In [159]:
def new_validation_cells(name,temp_flag):
    validation_mk_cell = nbf.v4.new_markdown_cell(f"#### {name} validation")
    if temp_flag:
        database = "__temp__"
    else:
        database = "fcsq"
    validation_code = f"""{name}_count = pydb.read_sql_query("SELECT count(*) as count from {database}.{name}\")
{name}_count"""
    validation_code_cell = nbf.v4.new_code_cell(validation_code)
    return [validation_mk_cell,validation_code_cell]

In [160]:
def add_table(name,number,desc,temp_flag,code):
    create_backup()
    cells = new_title_cell(number,name,desc)
    if temp_flag == False:
        cells = cells+new_drop_table_cells(name)        
    cells = cells+new_create_table_cells(name,code,temp_flag)+new_validation_cells(name,temp_flag)
    
    return cells



In [162]:
code = f"""
SELECT * FROM
fcsq.adopt_application_5
ORDER BY 
CASE_NUMBER, APP_DATE
"""

   
table_name,number,desc,temp_flag = get_inputs()

cells = add_table(table_name,number,desc,temp_flag,code)

nb['cells']= cells
with open('testing_notebook.ipynb', 'w') as f:
        nbf.write(nb, f)

table name (no spaces):  new
number:  4
table description:  vivrhe
Temporary table T/F:  F
